# Stance Detection Using Dummy Models

`Dummy Models`  
`AraStance Dataset`  
`Stance Detection` `Arabic Language`

---

In this notebook, we test the performance of dummy models as baselines for the task of article stance detection in the AraStance dataset. The dataset was introduced in the paper:
```
AraStance: A Multi-Country and Multi-Domain Dataset of Arabic Stance Detection for Fact Checking.
```

## Packages

In [ ]:
import numpy as np
from sklearn.metrics import f1_score
from utils import AraStanceData, stance_to_int

## Raw data

- Download the raw data:

In [ ]:
!wget https://github.com/Tariq60/arastance/archive/refs/heads/main.zip
!unzip /content/main.zip

- Let's start by reading the raw data:

In [ ]:
raw_train = AraStanceData("/content/arastance-main/data/train.jsonl")
raw_val = AraStanceData("/content/arastance-main/data/dev.jsonl")
raw_test = AraStanceData("/content/arastance-main/data/test.jsonl")

print(f'# training instances: {len(raw_train.stances)}')
print(f'# validation instances: {len(raw_val.stances)}')
print(f'# testing instances: {len(raw_test.stances)}')

# training instances: 2848
# validation instances: 569
# testing instances: 646


- Let's see an instance from the data:

In [ ]:
instance_no = 110
print(f"Claim text: {raw_train.claims[raw_train.article_claim[instance_no]]}")
print(f"Article text: {raw_train.articles[instance_no]}")
print(f"Stance: {raw_train.stances[instance_no]}")

Claim text: صدور بيان بقائمة أسماء ممنوعة ووجوب تغيير الاسم خلال ثلاثين يوم.
Article text: ماحقيقة الاسماء الممنوعة في السعودية ، سؤال تداوله الجميع عبر مواقع التواصل الاجتماعي بعد أن تناقل الكثيرون قائمة بأسماء منعتها الأحوال المدنية في السعودية شملت الكثير من الأسماء التي قد لايخلو منزل منها ، وتساءل البعض عن أسباب المنع وكيف يمكن تغيير الاسماء التي تم تسميتها مسبقا ؟ الأحوال المدنية توضح حقيقة الاسماء الممنوعة في السعودية أوضحت الأحوال المدنية السعودية أن ما يتم تناقله عبر المواقع الاجتماعية عن أسماء ممنوعة في السعودية هي معلومات خالية من الصحة ، وقالت عبر حسابها الرسمي على تويتر ( ما يتم تداوله حول صدور بيان بقائمة أسماء ممنوعة ووجوب تغيير الاسم خلال ثلاثين يوماً غير صحيح، ونأمل أخذ المعلومات من المصادر الرسمية. كما وأوضح المتحدث الرسمي للأحوال المدنية محمد الجاسر أنه لا يتم تسجيل أسماء المواليد غير الجائزة شرعا والأسماء غير الملائمة اجتماعيا ونحوها، وأضاف أن اللائحة التنفيذية لنظام الأحوال المدنية وضحت أنه لا يجوز التسمية بالأسماء التي نصت الفتاوى الشرعية على عدم جوازها، إضافة الأ

## Random classifier

- Let's start with the performance of a random classifier that gives random predictions:

In [ ]:
def random_classifier(labels, no_trials = 5):

  no_classes = len(np.unique(labels))
  accuracy, f1score, mf1score = [], [], []
  for _ in range(no_trials):
    predictions = np.array([np.random.randint(0, no_classes) for _ in range(len(labels))])

    acc = np.sum(predictions == np.array(labels)) / len(labels)
    accuracy.append(acc)

    f1 = f1_score(labels, predictions, average=None)
    f1score.append(f1)

    mf1 = f1_score(labels, predictions, average='macro')
    mf1score.append(mf1)

  return accuracy, f1score, mf1score

In [ ]:
val_labels = [stance_to_int[stance] for stance in raw_val.stances]
val_accuracy, val_f1score, val_mf1score= random_classifier(val_labels)
print("Validation Resutls:")
print("=====================")
print(f"Mean Accuracy: {np.mean(val_accuracy):.3f}")
agree, disagree, discuss, unrelated = np.mean(val_f1score, axis=0)
print("Mean Per Class F1 scores:")
print(f"Agree   : {agree:.3f}")
print(f"Disagree: {disagree:.3f}")
print(f"Discuss : {discuss:.3f}")
print(f"Unrelated: {unrelated:.3f}")
print(f"Mean Macro F1 scores: {np.mean(val_mf1score):.3f}")

Validation Resutls:
Mean Accuracy: 0.239
Mean Per Class F1 scores:
Agree   : 0.220
Disagree: 0.169
Discuss : 0.154
Unrelated: 0.327
Mean Macro F1 scores: 0.218


In [ ]:
test_labels = [stance_to_int[stance] for stance in raw_test.stances]
test_accuracy, test_f1score, test_mf1score= random_classifier(test_labels)
print("Testing Resutls:")
print("=====================")
print(f"Mean Accuracy: {np.mean(test_accuracy):.3f}")
agree, disagree, discuss, unrelated = np.mean(test_f1score, axis=0)
print("Mean Per Class F1 scores:")
print(f"Agree   : {agree:.3f}")
print(f"Disagree: {disagree:.3f}")
print(f"Discuss : {discuss:.3f}")
print(f"Unrelated: {unrelated:.3f}")
print(f"Mean Macro F1 scores: {np.mean(test_mf1score):.3f}")

Testing Resutls:
Mean Accuracy: 0.248
Mean Per Class F1 scores:
Agree   : 0.230
Disagree: 0.156
Discuss : 0.160
Unrelated: 0.336
Mean Macro F1 scores: 0.221


## Majority classifier

- Now, let's check the performance of a majority classifier that predicts the majority class all the time:

In [ ]:
def majority_classifier(labels, majority_class):

  predictions = [stance_to_int[majority_class]] * len(labels)
  accuracy = np.sum(np.array(predictions) == np.array(labels)) / len(labels)

  f1score = f1_score(labels, predictions, average=None)
  mf1score = f1_score(labels, predictions, average='macro')

  return accuracy, f1score, mf1score

In [ ]:
val_labels = [stance_to_int[stance] for stance in raw_val.stances]
val_accuracy, val_f1score, val_mf1score= majority_classifier(val_labels, 'unrelated')
print("Validation Resutls:")
print("=====================")
print(f"Mean Accuracy: {val_accuracy:.3f}")
agree, disagree, discuss, unrelated = val_f1score
print("Mean Per Class F1 scores:")
print(f"Agree   : {agree:.3f}")
print(f"Disagree: {disagree:.3f}")
print(f"Discuss : {discuss:.3f}")
print(f"Unrelated: {unrelated:.3f}")
print(f"Mean Macro F1 scores: {val_mf1score:.3f}")

Validation Resutls:
Mean Accuracy: 0.517
Mean Per Class F1 scores:
Agree   : 0.000
Disagree: 0.000
Discuss : 0.000
Unrelated: 0.681
Mean Macro F1 scores: 0.170


In [ ]:
test_labels = [stance_to_int[stance] for stance in raw_test.stances]
test_accuracy, test_f1score, test_mf1score = majority_classifier(test_labels, 'unrelated')
print("Testing Resutls:")
print("=====================")
print(f"Mean Accuracy: {test_accuracy:.3f}")
agree, disagree, discuss, unrelated = test_f1score
print("Mean Per Class F1 scores:")
print(f"Agree   : {agree:.3f}")
print(f"Disagree: {disagree:.3f}")
print(f"Discuss : {discuss:.3f}")
print(f"Unrelated: {unrelated:.3f}")
print(f"Mean Macro F1 scores: {test_mf1score:.3f}")

Testing Resutls:
Mean Accuracy: 0.554
Mean Per Class F1 scores:
Agree   : 0.000
Disagree: 0.000
Discuss : 0.000
Unrelated: 0.713
Mean Macro F1 scores: 0.178
